<a href="https://colab.research.google.com/github/afullhart/climateanalogs/blob/main/Figures/Zonal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reset -f

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import os
